# Importar librerias

In [3]:
import pandas as pd
import datetime
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

# Importamos nuestro input

In [4]:
#Archivo unificado y validado por cliente 
df = pd.read_csv("input.csv", sep=';', encoding ="latin-1")
df.head(5)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18808\97236701.py:2: DtypeWarning: Columns (20,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("input.csv", sep=';', encoding ="latin-1")


,DOCUMENTO,PRIORIDAD,MOTIVO_NO_PAGO,DESC_ACTIVIDAD,GENERO,EDAD,RANGO_HORA,SEGMENTO_CLIENTE,RIESGO_ACTUAL,RECUPERACION,...,PRIM_CTA,MORA_TEMPRANA,MANTENIDO,ACCION_MEJOR_CONTACTO,EFECTO_MEJOR_CONTACTO,NEGOCIACION_ACTIVA,CANTIDAD_GESTIONES,GESTION_LLAMADAS,CONTEO_CONTACTOS_DIRECTOS,MARCA_TIPO_CARTERA
0,11000000821,21,ASA_REDUCCION_INGRESOS,Â Asalariados.,F,23.0,Mañana,MONO_TC,NaN,NORMALIZADO,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9.0,1.0,1.0,PROD_DIG
1,11000000821,5,SIN_ACCESO_A_CANALES,Â Asalariados.,F,23.0,Mañana,MONO_TC,NaN,NORMALIZADO,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9.0,1.0,1.0,PROD_DIG
2,11000000821,5,ASA_REDUCCION_INGRESOS,Â Asalariados.,F,23.0,Tarde,MONO_TC,NaN,NORMALIZADO,...,X,X,X,HACER LLAMADA,YA PAGO,NO,9.0,1.0,1.0,PROD_DIG
3,11000000908,3,CALAMIDAD DOMESTICA,Â Asalariados.,M,23.0,Tarde,MONO_TC,10 ALTO,DETERIORADO,...,X,X,X,HACER LLAMADA,NO ATIENDE LLAMADA,NO,44.0,30.0,1.0,FGA
4,11000000908,16,CALAMIDAD DOMESTICA,Â Asalariados.,M,23.0,Mañana,MONO_TC,10 ALTO,DETERIORADO,...,X,X,X,HACER LLAMADA,NO ATIENDE LLAMADA,NO,44.0,30.0,1.0,FGA


# Análisis para cada caso particular

## Valores faltantes

In [8]:
missing_data = df.isnull().sum()
missing_data_percentage = (missing_data / len(df)) * 100
missing_data_summary = pd.concat([missing_data, missing_data_percentage], axis=1, keys=['Total', 'Porcentaje'])
missing_data_df = pd.DataFrame(missing_data_summary, columns=['Total', 'Porcentaje'])

# Agregar un mensaje antes del DataFrame
total_records = len(df)
message = f"De un total de {total_records} registros se encuentran los siguientes valores faltantes:\n"
print(message)
missing_data_df.head(50)

De un total de 723933 registros se encuentran los siguientes valores faltantes:



,Total,Porcentaje
DOCUMENTO,0,0.000000
PRIORIDAD,0,0.000000
MOTIVO_NO_PAGO,0,0.000000
DESC_ACTIVIDAD,0,0.000000
GENERO,0,0.000000
EDAD,372,0.051386
RANGO_HORA,0,0.000000
SEGMENTO_CLIENTE,0,0.000000
RIESGO_ACTUAL,133182,18.397006
RECUPERACION,0,0.000000


## Resumen estadistico

In [9]:
summary_statistics = df.describe()
summary_statistics.head()

,DOCUMENTO,PRIORIDAD,EDAD,MORA_INICIAL,MORA_ACTUAL,C_RIESGO,DIAS_ACTUAL,CICLO_OBLIG,CED_SIN_DIG,DIAS_OBLIG,DIAS_CL,CANTIDAD_GESTIONES,GESTION_LLAMADAS,CONTEO_CONTACTOS_DIRECTOS
count,7.239330e+05,723933.000000,723561.000000,723933.000000,723933.000000,723933.000000,593370.000000,723933.000000,7.239330e+05,723933.000000,723933.000000,723933.000000,723933.000000,723933.000000
mean,5.596959e+09,9.434281,40.341291,47.622902,47.195735,365.750753,45.502976,23.821544,5.557138e+08,34.248099,34.248099,20.879069,11.915929,1.663633
std,5.465188e+09,10.515613,13.474495,22.461669,33.453026,332.281866,26.738788,9.049222,5.119571e+08,23.987236,23.987236,33.406497,27.575819,2.279096
min,1.338300e+04,0.000000,-6.000000,30.000000,0.000000,-99.000000,1.000000,0.000000,3.383000e+03,3.000000,3.000000,0.000000,0.000000,0.000000
25%,1.430310e+08,3.000000,29.000000,30.000000,30.000000,0.000000,26.000000,16.000000,4.303099e+07,13.000000,13.000000,1.000000,0.000000,0.000000


## Cardinalidad de variables Categoricas

In [5]:
categorical_columns = df.select_dtypes(include=['object']).columns
categorical_cardinality = pd.DataFrame(columns=['Columna', 'Cardinalidad'])
for col in categorical_columns:
    cardinality = df[col].nunique()
    categorical_cardinality = categorical_cardinality.append({'Columna': col, 'Cardinalidad': cardinality}, ignore_index=True)
categorical_cardinality.head()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18808\3935950795.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  categorical_cardinality = categorical_cardinality.append({'Columna': col, 'Cardinalidad': cardinality}, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18808\3935950795.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  categorical_cardinality = categorical_cardinality.append({'Columna': col, 'Cardinalidad': cardinality}, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18808\3935950795.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  categorical_cardinality = categorical_cardinality.append({'Columna': col, 'Cardinalidad': cardinality}, ignore_index=True)
C:\Users\Usuario\Ap

,Columna,Cardinalidad
0,MOTIVO_NO_PAGO,124
1,DESC_ACTIVIDAD,481
2,GENERO,5
3,RANGO_HORA,2
4,SEGMENTO_CLIENTE,3


## Distribuciones sesgadas y transformaciones no lineales

In [6]:
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
skewed_distributions = pd.DataFrame(columns=['Columna', 'Skewness'])
for col in numeric_columns:
    skewness = skew(df[col])
    skewed_distributions = skewed_distributions.append({'Columna': col, 'Skewness': skewness}, ignore_index=True)
skewed_distributions.head()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18808\3718140062.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  skewed_distributions = skewed_distributions.append({'Columna': col, 'Skewness': skewness}, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18808\3718140062.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  skewed_distributions = skewed_distributions.append({'Columna': col, 'Skewness': skewness}, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18808\3718140062.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  skewed_distributions = skewed_distributions.append({'Columna': col, 'Skewness': skewness}, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18808\371814

,Columna,Skewness
0,DOCUMENTO,0.002484
1,PRIORIDAD,1.719999
2,EDAD,NaN
3,MORA_INICIAL,1.143838
4,MORA_ACTUAL,0.416355


## Desequilibrio en clases de variable objetivo

In [7]:
class_counts = df['RANGO_HORA'].value_counts()
class_imbalance = pd.DataFrame({'Clase': class_counts.index, 'Frecuencia': class_counts.values})
class_imbalance

,Clase,Frecuencia
0,Mañana,447411
1,Tarde,276522
